In [28]:
#https://covid19br.github.io/index.html

# Source: https://www.imperial.ac.uk/media/imperial-college/medicine/mrc-gida/2020-05-08-COVID19-Report-21.pdf

Start Day: Feb 24th
End Day: May 4th

Total Time: 70 days

Estimated attack rate: 3.5%

Start of intervention:

March 23th (day 28)

In [51]:
def plot_average_curve(sims, title, feature, absolute=False, capacity=False):
    fig = go.Figure()
      
    for col in [feature]:
        tmp=[sims[i][col] for i in range(len(sims))]
        infected_sims = pd.DataFrame(tmp).T.fillna(0)
        infected_sims['mean'] = infected_sims.apply(np.mean, axis=1)
        infected_sims['sd'] = infected_sims.drop(columns='mean').apply(np.std, axis=1)
        infected_sims = infected_sims.drop(columns=col)
    
    x = infected_sims.index.to_list()
    y = infected_sims['mean']
    sd = infected_sims['sd']
    
    if absolute: 
        y  =  y*12e6
        sd = sd*12e6

    fig.add_trace(go.Scatter(x=x, y=y,
        fill=None,
        mode='lines',
        line_color='indigo',
         name="Average Curve"
        ))

    fig.add_trace(go.Scatter(x=x, y=y+sd,
        fill=None,
        mode='lines',
        line_color='magenta',
        showlegend=False
       ))

    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        fill='tonexty', # fill area between trace0 and trace1
        mode='lines', line_color='indigo',      name="Standard Deviation"))

    fig.add_trace(go.Scatter(x=x, y=y-sd,
        fill=None,
        mode='lines',
        line_color='magenta',    showlegend=False
        ))


    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        fill='tonexty', # fill area between trace0 and trace1
        mode='lines', line_color='indigo',     showlegend=False
    ))
    
    if capacity:
        fig.add_trace(go.Scatter(
            x=x,
            y=len(x)*[0.0025],
            mode='lines', line_color='yellow', name='Capacity',    showlegend=True
        ))

    fig.update_layout(title = title)

    fig.show()

In [29]:
def simulate_pandemic(restrictions={'work':0, 'school': 0, 'home':0, 'neighbor':0},
                                  initial_infection=.0001, 
                                  p_r={'neighbor':.1, 'work':.3, 'school':.5, 'home':.7},
                                  lambda_leak=0, pop_size = None,
                                  graph_model = 'SP', file_path = None, 
                                  seed = None, it=None, policy=False):
    """
    Runs the course of the pandemic from the start until
    less than 1% of the population is simultaneously infected or no one is infected
    """
    np.random.seed(seed)
    
    G, data, status, pop = init_parameters(initial_infection, graph_model, file_path, 
                                                                    pop_size, seed)
    
    data_per_region = []
       
    zones = range(1, 343)
    
    policy_chosen = []
    
    infected_per_relation = {
        'home': 0,
        'work' : 0,
        'school': 0,
        'neighbor': 0
    }
    
    quarentine_restrictions = {
        'work':0.6, 'school': 1, 'home':0, 'neighbor':0.4
    }
    
    for day in tqdm(range(500)):
        
        if (status['removed']+status['susceptible'])>=pop:
            break
            
        update_graph(G)
       
        s, e, i, r, h, contacts_infected, status = get_time_series_row(G, pop)

        data.append([s, e, i, r, h, contacts_infected])
        
        
        if graph_model == 'SP':
            
            data_per_region.append(current_status_by_zone(G))
            
            #Lockdown
            if policy:
                if day >= 28:
                    restrictions = quarentine_restrictions

                policy_chosen.append(list(restrictions.values()))
            
            newly_infected = spread_one_step(G, p_r, restrictions, infected_per_relation, 
                                                lambda_leak, day)
        
            data[-1].append(newly_infected)
        
    columns = ['susceptible', 'exposed', 'infected', 'removed', 'hospitalized', 
               'contacts_infected_mean', 'newly_infected']

    time_series = pd.DataFrame(data, columns=columns)
    
    #return time_series, G, data_per_region, infected_per_relation, policy_chosen
    return time_series

In [31]:
sims_with_policy = Parallel(n_jobs=6)(delayed(simulate_pandemic)\
               (initial_infection = .0001, p_r = p_r, 
                policy=True, file_path='SP_multiGraph.gpickle') for i in range(20))

KeyboardInterrupt: 

In [29]:
sims_with_policies = [s for s in sims_with_policy if len(s) > 100]


[1 - s.iloc[70]['susceptible'] for s in sims_with_policies]

[0.03434729330353925,
 0.008685936711598119,
 0.02528292366467233,
 0.027697686152958978,
 0.03789735457363219,
 0.00737043177394936,
 0.02306638794781235,
 0.008307503784329318,
 0.0243999135010452,
 0.022075254090679786,
 0.03632956101780438,
 0.029355582786708023,
 0.053899661212427064,
 0.015876162329705235,
 0.01209183305701722,
 0.09066171700425285,
 0.020687666690694106,
 0.007046060693433276,
 0.009460823181719924]

In [31]:
plot_average_attack_curve(sims_with_policies, 'attack curve with policy')

In [41]:
def plot_average_infection_curve(sims, title):
    fig = go.Figure()
      
    for col in ['infected']:
        tmp=[sims[i][col] for i in range(len(sims))]
        infected_sims = pd.DataFrame(tmp).T.fillna(0)
        infected_sims['mean'] = infected_sims.apply(np.mean, axis=1)
        infected_sims['sd'] = infected_sims.drop(columns='mean').apply(np.std, axis=1)
        infected_sims = infected_sims.drop(columns=col)
    
    x = infected_sims.index.to_list()
    y = infected_sims['mean']*12e6
    sd = infected_sims['sd']*12e6

    fig.add_trace(go.Scatter(x=x, y=y,
        fill=None,
        mode='lines',
        line_color='indigo',
         name="Average Curve"
        ))

    fig.add_trace(go.Scatter(x=x, y=y+sd,
        fill=None,
        mode='lines',
        line_color='magenta',
        showlegend=False
       ))

    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        fill='tonexty', # fill area between trace0 and trace1
        mode='lines', line_color='indigo',      name="Standard Deviation"))

    fig.add_trace(go.Scatter(x=x, y=y-sd,
        fill=None,
        mode='lines',
        line_color='magenta',    showlegend=False
        ))


    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        fill='tonexty', # fill area between trace0 and trace1
        mode='lines', line_color='indigo',     showlegend=False
    ))

    fig.update_layout(title = title)

    fig.show()

In [36]:
plot_average_infection_curve(sims_with_policies, 'infection curive with policies')

In [37]:
import pickle as pkl
with open('simulations_with_policy.pkl', 'wb') as f:
    pkl.dump(sims_with_policies, f)

In [3]:
import pickle as pkl
sims = pkl.load(open('simulations_with_policy.pkl', 'rb'))

In [25]:
def plot_average_hospitalized_curve(sims, title):
    fig = go.Figure()
      
    for col in ['hospitalized']:
        tmp=[sims[i][col] for i in range(len(sims))]
        infected_sims = pd.DataFrame(tmp).T.fillna(0)
        infected_sims['mean'] = infected_sims.apply(np.mean, axis=1)
        infected_sims['sd'] = infected_sims.drop(columns='mean').apply(np.std, axis=1)
        infected_sims = infected_sims.drop(columns=col)
    
    x = infected_sims.index.to_list()
    y = infected_sims['mean']
    sd = infected_sims['sd']

    fig.add_trace(go.Scatter(x=x, y=y,
        fill=None,
        mode='lines',
        line_color='indigo',
         name="Average Curve"
        ))

    fig.add_trace(go.Scatter(x=x, y=y+sd,
        fill=None,
        mode='lines',
        line_color='magenta',
        showlegend=False
       ))

    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        fill='tonexty', # fill area between trace0 and trace1
        mode='lines', line_color='indigo',      name="Standard Deviation"))

    fig.add_trace(go.Scatter(x=x, y=y-sd,
        fill=None,
        mode='lines',
        line_color='magenta',    showlegend=False
        ))


    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        fill='tonexty', # fill area between trace0 and trace1
        mode='lines', line_color='indigo',     showlegend=False
    ))
    
    fig.add_trace(go.Scatter(
        x=x,
        y=len(x)*[0.0025],
        mode='lines', line_color='yellow', name='Capacity',    showlegend=True
    ))

    fig.update_layout(title = title)

    fig.show()

In [35]:
plot_average_hospitalized_curve(sims, 'average_hospitalized_curve')
# Source of compartive data
#https://github.com/covid19br/nowcasting/blob/master/dados_processados/projecao_leitos/municipios/SP/Sao_Paulo/hospitalizados/hopitalized_2020-05-20.csv

In [33]:
def simulate_pandemic(restrictions={'work':0, 'school': 0, 'home':0, 'neighbor':0},
                                  initial_infection=.0001, 
                                  p_r={'neighbor':.1, 'work':.3, 'school':.5, 'home':.7},
                                  lambda_leak=0, pop_size = None,
                                  graph_model = 'SP', file_path = None, 
                                  seed = None, it=None, policy=False):
    """
    Runs the course of the pandemic from the start until
    less than 1% of the population is simultaneously infected or no one is infected
    """
    np.random.seed(seed)
    
    G, data, status, pop = init_parameters(initial_infection, graph_model, file_path, 
                                                                    pop_size, seed)
    
    data_per_region = []
       
    zones = range(1, 343)
    
    policy_chosen = []
    
    infected_per_relation = {
        'home': 0,
        'work' : 0,
        'school': 0,
        'neighbor': 0
    }
    
    quarentine_restrictions = {
        'work':0.6, 'school': 1, 'home':0, 'neighbor':0.4
    }
    
    regular_restrictions = {
        'work':0, 'school': 0, 'home':0, 'neighbor':0
    }
    
    for day in tqdm(range(500)):
        
        if (status['removed']+status['susceptible'])>=pop:
            break
            
        update_graph(G)
       
        s, e, i, r, h, contacts_infected, status = get_time_series_row(G, pop)

        data.append([s, e, i, r, h, contacts_infected])
        
        
        if graph_model == 'SP':
            
            data_per_region.append(current_status_by_zone(G))
            
            #Lockdown
            if policy:
                if day >= 28:
                    restrictions = quarentine_restrictions

                if day >= 97:
                    restrictions = regular_restrictions
                policy_chosen.append(list(restrictions.values()))
                
            newly_infected = spread_one_step(G, p_r, restrictions, infected_per_relation, 
                                                lambda_leak, day)
        
            data[-1].append(newly_infected)
        
    columns = ['susceptible', 'exposed', 'infected', 'removed', 'hospitalized', 
               'contacts_infected_mean', 'newly_infected']

    time_series = pd.DataFrame(data, columns=columns)
    
    #return time_series, G, data_per_region, infected_per_relation, policy_chosen
    return time_series

In [34]:
sims_end_policy_may = Parallel(n_jobs=6)(delayed(simulate_pandemic)\
               (initial_infection = .0001, p_r = p_r, 
                policy=True, file_path='SP_multiGraph.gpickle') for i in range(20))

In [36]:
with open('simulations_end_policy_may.pkl', 'wb') as f:
    pkl.dump(sims_end_policy_may, f)

In [37]:
plot_average_hospitalized_curve(sims_end_policy_may, 'average_hospitalized_curve')


In [42]:
plot_average_infection_curve(sims_end_policy_may, 'sims_wnd_policy_may')

In [46]:
plot_average_infection_curve(sims, 'sims_with_policy')

In [47]:
plot_average_hospitalized_curve(sims, 'sims_with_policy')

In [44]:
sims_no_policy = Parallel(n_jobs=6)(delayed(simulate_pandemic)\
               (initial_infection = .0001, p_r = p_r, 
                policy=False, file_path='SP_multiGraph.gpickle') for i in range(20))

In [49]:
with open('simulations_no_policy.pkl', 'wb') as f:
    pkl.dump(sims_no_policy, f)

In [45]:
plot_average_infection_curve(sims_no_policy, 'sims_no_policy')

In [48]:
plot_average_hospitalized_curve(sims_no_policy, 'sims_no_policy')

In [64]:
for s in sims_no_policy:
    s['R_0'] = s['newly_infected'] / (s['infected']*55000)

In [67]:
plot_average_curve(sims_no_policy, 'R_0', 'R_0')

In [74]:
plot_average_curve(sims_end_policy_may, 'contacts_infected_mean', 'contacts_infected_mean')